In [ ]:
!pip install gradio --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/FinalProject/

/content/drive/.shortcut-targets-by-id/1jSrOhQqocwt88cBXGRwOFWQ8gKv3DI8i/FinalProject


In [ ]:
import gradio as gr
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image
from tensorflow import keras
import cv2
import os

# Directory containing the model file
dcnn = "/content/drive/MyDrive/FinalProject/base no canny.keras"
encoder = "/content/drive/MyDrive/FinalProject/encoder.keras"
dcnn_canny = "/content/drive/MyDrive/FinalProject/0.25.keras"
encoder_canny = "/content/drive/MyDrive/FinalProject/encoder_x.keras"

# Load the models
dcnn_xception_model = load_model(dcnn)
encoder_decoder_model = load_model(encoder)
dcnn_xception_canny_model = load_model(dcnn_canny)
encoder_decoder_canny_model = load_model(encoder_canny)

# Validate the image extension to ensure it's PNG, JPG, or JPEG
def validate_image(image_path):
    allowed_extensions = ['.png', '.jpg', '.jpeg']
    _, ext = os.path.splitext(image_path)
    return ext.lower() in allowed_extensions

# Apply pre-processing
def apply_canny_and_blend(image_path, alpha=0.25):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Invalid image. Ensure the file path is correct and the file is an image.")

    # Convert image to grayscale for Canny
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 65, 100)  # Perform Canny edge detection

    # Convert edges to 3-channel image
    edges_3_channel = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

    # Resize edges to match the original image (if necessary)
    if image.shape != edges_3_channel.shape:
        edges_3_channel = cv2.resize(edges_3_channel, (image.shape[1], image.shape[0]))

    # Perform alpha blending
    blended_image = cv2.addWeighted(image, 1 - alpha, edges_3_channel, alpha, 0)
    return blended_image

# Predicting the images
def predict(image, model_choice, canny_choice):
    if not image:
        return "Please upload an image."

    try:
        # Validate image extension
        if not validate_image(image):
            return "Invalid file type. Allowed file types: PNG, JPG, JPEG."

        # Apply Canny edge detection and blending if selected
        if canny_choice == "With Canny":
            blended_image = apply_canny_and_blend(image)
            img = Image.fromarray(cv2.cvtColor(blended_image, cv2.COLOR_BGR2RGB))
        else:
            img = Image.open(image).convert("RGB")  # Ensure 3 channels for RGB if not selected

        # Resize image based on selected model
        if model_choice == "DCNN-Xception":
            img = img.resize((299, 299))  # Resize to 299x299 for DCNN-Xception
            model = dcnn_xception_canny_model if canny_choice == "With Canny" else dcnn_xception_model
        elif model_choice == "Encoder-Decoder":
            img = img.resize((224, 224))  # Resize to 224x224 for Encoder-Decoder
            model = encoder_decoder_canny_model if canny_choice == "With Canny" else encoder_decoder_model
        else:
            return "Invalid model choice."

        # Convert image to array
        img_array = keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)  # Add batch dimension

        # Make predictions using the selected model
        predictions = model.predict(img_array)

        # Decode predictions into percentages
        negative, positive = tuple(predictions[0])

        # Determine the class with the highest value and show output
        if positive > negative:
            return f"This image is positive ({positive * 100:.2f}% confidence). Suggestion: Parasite found. Consider further investigation."
        else:
            return f"This image is negative ({negative * 100:.2f}% confidence). No parasite detected."

    except Exception as e:
        return f"Error processing image: {e}"

# Model and Canny choices for user
model_choices = ["---", "DCNN-Xception", "Encoder-Decoder"]
canny_choices = ["Without Canny", "With Canny"]

#  GUI Interface
with gr.Blocks(theme=gr.themes.Citrus()) as demo:
    gr.Markdown("""
    <link href="https://fonts.googleapis.com/css2?family=Noto+Sans:ital,wght@0,100..900;1,100..900&display=swap" rel="stylesheet">
    <div style="font-family: 'Noto Sans', sans-serif; text-align: center; font-size: 24px;">
        <strong>Detection of <em>Trichomonas vaginalis</em> in Microscopic Images Using DCNN-Xception with Canny Edge Detection</strong>
    </div>
    """)

    gr.Markdown("""
    <link href="https://fonts.googleapis.com/css2?family=Noto+Sans:ital,wght@0,100..900;1,100..900&display=swap" rel="stylesheet">
    <div style="font-family: 'Noto Sans', sans-serif; text-align: center; font-size: 16px;">
        University of Santo Tomas College of Information and Computing Sciences<br>
        Guevarra, Javier, Tapao (2024)
    </div>
    """)

    # GUI components
    with gr.Row():
        with gr.Column(scale=2, min_width=300):
            model_dropdown = gr.Dropdown(choices=model_choices, label="Select Model Architecture")
            image_input = gr.Image(label="Upload Image File", type="filepath", interactive=True)
            gr.Markdown("""
            <link href="https://fonts.googleapis.com/css2?family=Noto+Sans:ital,wght@0,100..900;1,100..900&display=swap" rel="stylesheet">
            <div style="font-family: 'Noto Sans', sans-serif; text-align: center; font-size: 12px; color: grey;">
                <strong>Allowed file types:</strong> .PNG, .JPG, .JPEG
            </div>
            """)

        with gr.Column(scale=1, min_width=800):
            canny_toggle = gr.Radio(choices=canny_choices, label="Canny Edge Detection")
            output = gr.Textbox(label="Prediction")
            submit_button = gr.Button("Submit")
            reset_button = gr.Button("Reset")

        # Button actions
        submit_button.click(predict, inputs=[image_input, model_dropdown, canny_toggle], outputs=output)
        reset_button.click(
            lambda: ("", None, "", ""),  # Reset values for model_dropdown, image_input, and canny_toggle
            inputs=[],
            outputs=[model_dropdown, image_input, canny_toggle, output]
        )

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://65ae29076938c5ecd6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
pip install gradio pyinstaller tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 710.1/710.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.4/338.4 kB 23.6 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image
from tensorflow import keras
import cv2
import os
import sys

# Directory containing the model file (will be bundled by PyInstaller)
dcnn = "base no canny.keras"
encoder = "encoder.keras"
dcnn_canny = "0.25.keras"
encoder_canny = "encoder_x.keras"

# Ensure models are loaded properly when running as an executable
def load_models():
    base_path = getattr(sys, '_MEIPASS', os.getcwd())

    # Update paths based on whether running as a bundled app or directly
    dcnn_model = load_model(os.path.join(base_path, dcnn))
    encoder_model = load_model(os.path.join(base_path, encoder))
    dcnn_canny_model = load_model(os.path.join(base_path, dcnn_canny))
    encoder_canny_model = load_model(os.path.join(base_path, encoder_canny))

    return dcnn_model, encoder_model, dcnn_canny_model, encoder_canny_model

dcnn_xception_model, encoder_decoder_model, dcnn_xception_canny_model, encoder_decoder_canny_model = load_models()

# Validate the image extension to ensure it's PNG, JPG, or JPEG
def validate_image(image_path):
    allowed_extensions = ['.png', '.jpg', '.jpeg']
    _, ext = os.path.splitext(image_path)
    return ext.lower() in allowed_extensions

# Apply pre-processing
def apply_canny_and_blend(image_path, alpha=0.25):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Invalid image. Ensure the file path is correct and the file is an image.")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 65, 100)

    edges_3_channel = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

    if image.shape != edges_3_channel.shape:
        edges_3_channel = cv2.resize(edges_3_channel, (image.shape[1], image.shape[0]))

    blended_image = cv2.addWeighted(image, 1 - alpha, edges_3_channel, alpha, 0)
    return blended_image

# Predicting the images
def predict(image, model_choice, canny_choice):
    if not image:
        return "Please upload an image."

    try:
        if not validate_image(image):
            return "Invalid file type. Allowed file types: PNG, JPG, JPEG."

        if canny_choice == "With Canny":
            blended_image = apply_canny_and_blend(image)
            img = Image.fromarray(cv2.cvtColor(blended_image, cv2.COLOR_BGR2RGB))
        else:
            img = Image.open(image).convert("RGB")

        if model_choice == "DCNN-Xception":
            img = img.resize((299, 299))
            model = dcnn_xception_canny_model if canny_choice == "With Canny" else dcnn_xception_model
        elif model_choice == "Encoder-Decoder":
            img = img.resize((224, 224))
            model = encoder_decoder_canny_model if canny_choice == "With Canny" else encoder_decoder_model
        else:
            return "Invalid model choice."

        img_array = keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)

        predictions = model.predict(img_array)

        negative, positive = tuple(predictions[0])

        if positive > negative:
            return f"This image is positive ({positive * 100:.2f}% confidence). Suggestion: Parasite found. Consider further investigation."
        else:
            return f"This image is negative ({negative * 100:.2f}% confidence). No parasite detected."

    except Exception as e:
        return f"Error processing image: {e}"

# Model and Canny choices for user
model_choices = ["---", "DCNN-Xception", "Encoder-Decoder"]
canny_choices = ["Without Canny", "With Canny"]

#  GUI Interface
with gr.Blocks(theme=gr.themes.Citrus()) as demo:
    gr.Markdown("""
    <div style="font-family: 'Noto Sans', sans-serif; text-align: center; font-size: 24px;">
        <strong>Detection of <em>Trichomonas vaginalis</em> in Microscopic Images Using DCNN-Xception with Canny Edge Detection</strong>
    </div>
    """)

    gr.Markdown("""
    <link href="https://fonts.googleapis.com/css2?family=Noto+Sans:ital,wght@0,100..900;1,100..900&display=swap" rel="stylesheet">
    <div style="font-family: 'Noto Sans', sans-serif; text-align: center; font-size: 16px;">
        University of Santo Tomas College of Information and Computing Sciences<br>
        Guevarra, Javier, Tapao (2024)
    </div>
    """)

    # GUI components
    with gr.Row():
        with gr.Column(scale=2, min_width=300):
            model_dropdown = gr.Dropdown(choices=model_choices, label="Select Model Architecture")
            image_input = gr.Image(label="Upload Image File", type="filepath", interactive=True)
            gr.Markdown("""
            <div style="font-family: 'Noto Sans', sans-serif; text-align: center; font-size: 12px; color: grey;">
                <strong>Allowed file types:</strong> .PNG, .JPG, .JPEG
            </div>
            """)

        with gr.Column(scale=1, min_width=800):
            canny_toggle = gr.Radio(choices=canny_choices, label="Canny Edge Detection")
            output = gr.Textbox(label="Prediction")
            submit_button = gr.Button("Submit")
            reset_button = gr.Button("Reset")

        submit_button.click(predict, inputs=[image_input, model_dropdown, canny_toggle], outputs=output)
        reset_button.click(
            lambda: ("", None, "", ""),
            inputs=[],
            outputs=[model_dropdown, image_input, canny_toggle, output]
        )

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bef66e32e933d3fcbe.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip uninstall pathlib

Found existing installation: pathlib 1.0.1
Uninstalling pathlib-1.0.1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/pathlib-1.0.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/pathlib.py
Proceed (Y/n)? Y
  Successfully uninstalled pathlib-1.0.1


In [ ]:
!pyinstaller --onefile --windowed --add-data "base_no_canny.keras;." --add-data "encoder.keras;." --add-data "0.25.keras;." --add-data "encoder_x.keras;." app.py

usage: pyinstaller [-h] [-v] [-D] [-F] [--specpath DIR] [-n NAME]
                   [--contents-directory CONTENTS_DIRECTORY] [--add-data SOURCE:DEST]
                   [--add-binary SOURCE:DEST] [-p DIR] [--hidden-import MODULENAME]
                   [--collect-submodules MODULENAME] [--collect-data MODULENAME]
                   [--collect-binaries MODULENAME] [--collect-all MODULENAME]
                   [--copy-metadata PACKAGENAME] [--recursive-copy-metadata PACKAGENAME]
                   [--additional-hooks-dir HOOKSPATH] [--runtime-hook RUNTIME_HOOKS]
                   [--exclude-module EXCLUDES] [--splash IMAGE_FILE]
                   [-d {all,imports,bootloader,noarchive}] [--optimize LEVEL]
                   [--python-option PYTHON_OPTION] [-s] [--noupx] [--upx-exclude FILE] [-c] [-w]
                   [--hide-console {hide-late,hide-early,minimize-early,minimize-late}]
                   [-i <FILE.ico or FILE.exe,ID or FILE.icns or Image or "NONE">]
                 

In [ ]:
import os
os.chdir('/content/drive/My Drive/FinalProject/') # Adjust the path if necessary

In [ ]:
import os
# Construct the add-data arguments with os.pathsep and escaping potential spaces
add_data_args = [
    f'--add-data={os.path.abspath("base_no_canny.keras")}{os.pathsep}.',
    f'--add-data={os.path.abspath("encoder.keras")}{os.pathsep}.',
    f'--add-data={os.path.abspath("0.25.keras")}{os.pathsep}.',
    f'--add-data={os.path.abspath("encoder_x.keras")}{os.pathsep}.',
]

# Build the full PyInstaller command
pyinstaller_cmd = [
    'pyinstaller',
    '--onefile',
    '--windowed',
    *add_data_args, # Unpack the add_data arguments
    '--distpath=/content/drive/My Drive/FinalProject/',  # Specify output path
    'app.py'
]

# Execute the command
os.system(' '.join(pyinstaller_cmd))

256

In [ ]:
new_folder_path = 'Executable'  # Or any name you prefer
import os
os.makedirs(new_folder_path, exist_ok=True)

import os
# ... (add_data_args remain the same)

# Build the PyInstaller command with --distpath
pyinstaller_cmd = [
    'pyinstaller',
    '--onefile',
    '--windowed',
    *add_data_args,
    f'--distpath={new_folder_path}',  # Save in the new folder
    'app.py'
]

# Execute the command
os.system(' '.join(pyinstaller_cmd))

256

In [ ]:
# Create a simple Python script named test.py:
print("Hello, world!")

# Run PyInstaller:
!pyinstaller --onefile test.py

Hello, world!
306 INFO: PyInstaller: 6.11.1, contrib hooks: 2024.10
306 INFO: Python: 3.10.12
308 INFO: Platform: Linux-6.1.85+-x86_64-with-glibc2.35
308 INFO: Python environment: /usr
Script file 'test.py' does not exist.


In [ ]:
import os
print(os.getcwd())

/content/drive/.shortcut-targets-by-id/1jSrOhQqocwt88cBXGRwOFWQ8gKv3DI8i/FinalProject
